In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

intakes_file_path = os.path.join("center_data", "Austin_Animal_Center_Intakes.csv")

dog_intakes_df = pd.read_csv(intakes_file_path)\
    .rename(columns = lambda df: df.lower().replace(" ", "_"))\
    .drop("monthyear", axis=1)

dog_intakes_df = dog_intakes_df[dog_intakes_df["animal_type"] == 'Dog']\
    .rename(columns = {
    "intake_type":"type",
    "sex_upon_intake":"sex",
    "age_upon_intake":"age"
    })\
    .assign(is_intake=True)\
    .assign(datetime = lambda df: pd.to_datetime(df["datetime"]))\
    .reset_index(drop=True)

# dogs_intakes_df.to_csv('dog_intakes.csv')
dog_intakes_df.head(5)

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake
0,A786884,*Brock,2019-01-03 16:19:00,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,True
1,A706918,Belle,2015-07-05 12:59:00,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,True
2,A724273,Runster,2016-04-14 18:43:00,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,True
3,A682524,Rio,2014-06-29 10:38:00,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,True
4,A743852,Odin,2017-02-18 12:46:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,2 years,Labrador Retriever Mix,Chocolate,True


In [2]:
outcomes_file_path = os.path.join("center_data", "Austin_Animal_Center_Outcomes.csv")

dog_outcomes_df = pd.read_csv(outcomes_file_path)\
    .rename(columns = lambda df: df.lower().replace(" ", "_"))\
    .drop("monthyear", axis=1)

dog_outcomes_df = dog_outcomes_df[
    (dog_outcomes_df["animal_type"] == 'Dog') & (dog_outcomes_df['outcome_type'] == 'Adoption')]\
    .rename(columns = {
        "outcome_type":"type",
        "sex_upon_outcome":"sex",
        "age_upon_outcome":"age"
        })\
    .assign(is_intake=False)\
    .assign(datetime = lambda df: pd.to_datetime(df["datetime"]))\
    .reset_index(drop=True)    

# dogs_intakes_df.to_csv('dog_intakes.csv')
dog_outcomes_df.head(5)

,animal_id,name,datetime,date_of_birth,type,outcome_subtype,animal_type,sex,age,breed,color,is_intake
0,A789027,Lennie,2019-02-17 11:44:00,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream,False
1,A720371,Moose,2016-02-13 17:59:00,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,False
2,A789027,Lennie,2019-03-10 12:25:00,02/13/2017,Adoption,NaN,Dog,Neutered Male,2 years,Chihuahua Shorthair Mix,Cream,False
3,A794494,Zoey,2019-05-14 16:27:00,06/14/2018,Adoption,NaN,Dog,Spayed Female,10 months,Wire Hair Fox Terrier Mix,White/Black,False
4,A764361,Astro,2018-01-03 19:15:00,12/28/2016,Adoption,NaN,Dog,Neutered Male,1 year,Norwich Terrier Mix,Black/Tan,False


In [3]:
dog_concat_df = pd\
    .concat([dog_intakes_df, dog_outcomes_df], sort=False)\
    .sort_values(by=["animal_id", "datetime"])\
     [lambda df: df.duplicated(["animal_id"],keep=False)]\
    .fillna("N/A")
# dog_concat_df[dog_concat_df["animal_id"] == "A047759"]

dog_concat_df.head()

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake,date_of_birth,outcome_subtype
39209,A006100,Scamp,2014-03-07 14:26:00,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White,True,N/A,N/A
1859,A006100,Scamp,2014-12-19 10:21:00,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White,True,N/A,N/A
8663,A006100,Scamp,2017-12-07 14:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,True,N/A,N/A
45817,A200922,Carlos,2013-10-03 15:47:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,True,N/A,N/A
5096,A200922,Carlos,2013-11-22 09:44:00,N/A,Adoption,N/A,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,False,10/03/1997,Foster


In [8]:
intakes_agg = dog_concat_df[dog_concat_df["is_intake"]].groupby("animal_id")["animal_id"].count()
outcomes_agg = dog_concat_df[~dog_concat_df["is_intake"]].groupby("animal_id")["animal_id"].count()

In [6]:
def qualified(num_intakes, num_outcomes):
    is_qualified = False
    
    if num_intakes == num_outcomes or num_intakes-1 == num_outcomes:
        is_qualified=True
    
    return is_qualified

In [9]:
validation_agg = pd.merge(
    intakes_agg.to_frame().rename(columns = {"animal_id":"intake_count"}),
    outcomes_agg.to_frame().rename(columns = {"animal_id":"outcome_count"}),
    how="outer",
    left_index=True,
    right_index=True
).dropna().reset_index()

validation_agg["valid"] = np.vectorize(qualified)(validation_agg["intake_count"], validation_agg["outcome_count"])
matched_stays_df = validation_agg[validation_agg["valid"]]["animal_id"]

0        A200922
1        A210457
2        A226069
3        A249087
4        A274546
          ...   
26226    A811450
26227    A811507
26228    A811509
26229    A811534
26230    A811598
Name: animal_id, Length: 25802, dtype: object

In [10]:
working_df = dog_concat_df.loc[dog_concat_df["animal_id"].isin(matched_stays_df.values)].reset_index(drop=True)
working_df.head(5)

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake,date_of_birth,outcome_subtype
0,A200922,Carlos,2013-10-03 15:47:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,True,N/A,N/A
1,A200922,Carlos,2013-11-22 09:44:00,N/A,Adoption,N/A,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,False,10/03/1997,Foster
2,A210457,Caleb,2016-09-28 12:05:00,4424 S Mopac #412 in Austin (TX),Public Assist,Aged,Dog,Neutered Male,17 years,Chihuahua Shorthair,Tan/Black,True,N/A,N/A
3,A210457,Caleb,2016-10-07 12:34:00,N/A,Adoption,N/A,Dog,Neutered Male,17 years,Chihuahua Shorthair,Tan/Black,False,06/01/1999,Foster
4,A226069,Cedar,2015-10-06 12:29:00,Wheless Ln And Berkman Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,15 years,Labrador Retriever/Beagle,Sable/White,True,N/A,N/A


In [11]:
# Color

def split_colors(color,*,primary=True):
    split_char = "/"
    
    color_list = color.split(split_char) if color.find(split_char) >=0 else color

    color_output = None
    if primary:
        color_output = color_list[0] if type(color_list) == list else color_list
    else:
        color_output = color_list[1] if type(color_list) == list else None
    
    return color_output

def convert_color(color, conversion_dict):
    converted_color = None
    for key, value in conversion_dict.items():
        if not color == None:
            if color in value:
                converted_color = key
        else:
            converted_color = None
    return converted_color


def total_color_set(color_series):
    
    primary_secondary = [
        split_colors(color)
        for color 
        in color_series.unique()
    ]

    primary = {pair[0] if type(pair) == list else pair for pair in primary_secondary}
    secondary = {pair[1] for pair in primary_secondary if type(pair) == list}

    return primary|secondary
    
total_colors = total_color_set(working_df["color"])

color_conversion_dict = {
    "Black":[color for color in total_colors if color.startswith("Black")],
    "Brown":[color for color in total_colors if color.startswith("Brown")],
    "Blue":[color for color in total_colors if color.startswith("Blue")],
    "Red":[color for color in total_colors if color.startswith("Red")],
    "Yellow":[color for color in total_colors if color.startswith("Yellow")],
    "Gray":["Gray", "Silver"],
    "Tan":["Tan", "Buff", "Orange", "Gold", "Fawn", "Apricot"],
    "White":["White", "Cream"],
    "Tricolor":["Tricolor", "Calico", "Sable"],
    "Pink": "Pink"
}
color_conversion_dict["Red"].append("Ruddy")
color_conversion_dict["Brown"].extend(["Agouti", "Chocolate", "Liver", "Liver Tick"])

{'Black': ['Black Smoke', 'Black Tiger', 'Black', 'Black Brindle'],
 'Brown': ['Brown Merle',
  'Brown',
  'Brown Brindle',
  'Brown Tiger',
  'Agouti',
  'Chocolate',
  'Liver',
  'Liver Tick'],
 'Blue': ['Blue Cream',
  'Blue Tick',
  'Blue Tiger',
  'Blue',
  'Blue Merle',
  'Blue Smoke'],
 'Red': ['Red Tick', 'Red', 'Red Merle', 'Ruddy'],
 'Yellow': ['Yellow', 'Yellow Brindle'],
 'Gray': ['Gray', 'Silver'],
 'Tan': ['Tan', 'Buff', 'Orange', 'Gold', 'Fawn', 'Apricot'],
 'White': ['White', 'Cream'],
 'Tricolor': ['Tricolor', 'Calico', 'Sable'],
 'Pink': 'Pink'}

In [12]:
working_df["color_primary"] = working_df["color"]\
    .apply(split_colors)\
    .apply(lambda color: convert_color(color, color_conversion_dict))

working_df["color_secondary"] = working_df["color"]\
    .apply(lambda color: split_colors(color, primary=False))\
    .apply(lambda color: convert_color(color, color_conversion_dict))

working_df = working_df.drop("color", axis=1)
# Do not run this cell again or else drop won't work

In [13]:
working_df.loc[working_df['breed'].str.contains('Labrador Retriever/'), 'breed'] = 'Labrador Retriever Mix'
working_df.loc[working_df['breed'].str.contains('Chihuahua Shorthair/'), 'breed'] = 'Chihuahua Shorthair Mix'
working_df.loc[working_df['breed'].str.contains('Pit Bull/'), 'breed'] = 'Pit Bull Mix'
working_df.loc[working_df['breed'].str.contains('German Shepherd/'), 'breed'] = 'German Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('Australian Cattle Dog/'), 'breed'] = 'Australian Cattle Dog Mix'
working_df.loc[working_df['breed'].str.contains('Dachshund/'), 'breed'] = 'Dachshund Mix'
working_df.loc[working_df['breed'].str.contains('Border Collie/'), 'breed'] = 'Border Collie Mix'
working_df.loc[working_df['breed'].str.contains('Boxer/'), 'breed'] = 'Boxer Mix'
working_df.loc[working_df['breed'].str.contains('Catahoula/'), 'breed'] = 'Catahoula Mix'
working_df.loc[working_df['breed'].str.contains('Miniature Poodle/'), 'breed'] = 'Miniature Poodle Mix'
working_df.loc[working_df['breed'].str.contains('Australian Shepherd/'), 'breed'] = 'Australian Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('Pointer/'), 'breed'] = 'Pointer Mix'
working_df.loc[working_df['breed'].str.contains('Jack Russell Terrier/'), 'breed'] = 'Jack Russell Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Beagle/'), 'breed'] = 'Beagle Mix'
working_df.loc[working_df['breed'].str.contains('Yorkshire Terrier/'), 'breed'] = 'Yorkshire Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Rat Terrier/'), 'breed'] = 'Rat Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Cairn Terrier/'), 'breed'] = 'Cairn Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Staffordshire/'), 'breed'] = 'Staffordshire Mix'
working_df.loc[working_df['breed'].str.contains('Great Pyrenees/'), 'breed'] = 'Great Pyrenees Mix'
working_df.loc[working_df['breed'].str.contains('Miniature Schnauzer/'), 'breed'] = 'Miniature Schnauzer Mix'
working_df.loc[working_df['breed'].str.contains('Siberian Husky/'), 'breed'] = 'Siberian Husky Mix'
working_df.loc[working_df['breed'].str.contains('Chihuahua Longhair/'), 'breed'] = 'Chihuahua Longhair Mix'
working_df.loc[working_df['breed'].str.contains('Anatol Shepherd/'), 'breed'] = 'Anatol Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('Plott Hound/'), 'breed'] = 'Plott Hound Mix'
working_df.loc[working_df['breed'].str.contains('Black Mouth Cur/'), 'breed'] = 'Black Mouth Cur Mix'
working_df.loc[working_df['breed'].str.contains('Rottweiler/'), 'breed'] = 'Rottweiler Mix'
working_df.loc[working_df['breed'].str.contains('Australian Kelpie/'), 'breed'] = 'Australian Kelpie Mix'
working_df.loc[working_df['breed'].str.contains('American Bulldog/'), 'breed'] = 'American Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Miniature Pinscher/'), 'breed'] = 'Miniature Pinscher Mix'
working_df.loc[working_df['breed'].str.contains('Border Terrier/'), 'breed'] = 'Border Terrier Mix'
working_df.loc[working_df['breed'].str.contains('American Pit Bull Terrier/'), 'breed'] = 'American Pit Bull Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Cardigan Welsh Corgi/'), 'breed'] = 'Cardigan Welsh Corgi Mix'
working_df.loc[working_df['breed'].str.contains('Golden Retriever/'), 'breed'] = 'Golden Retriever Mix'
working_df.loc[working_df['breed'].str.contains('Maltese/'), 'breed'] = 'Maltese Mix'
working_df.loc[working_df['breed'].str.contains('Blue Lacy/'), 'breed'] = 'Blue Lacy Mix'
working_df.loc[working_df['breed'].str.contains('Queensland Heeler/'), 'breed'] = 'Queensland Heeler Mix'
working_df.loc[working_df['breed'].str.contains('American Staffordshire Terrier/'), 'breed'] = 'American Staffordshire Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Basset Hound/'), 'breed'] = 'Basset Hound Mix'
working_df.loc[working_df['breed'].str.contains('Manchester Terrier/'), 'breed'] = 'Manchester Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Black/Tan Hound/'), 'breed'] = 'Black/Tan Hound Mix'
working_df.loc[working_df['breed'].str.contains('Collie Smooth/'), 'breed'] = 'Collie Smooth Mix'
working_df.loc[working_df['breed'].str.contains('Norfolk Terrier/'), 'breed'] = 'Norfolk Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Dachshund Longhair/'), 'breed'] = 'Dachshund Longhair Mix'
working_df.loc[working_df['breed'].str.contains('Redbone Hound/'), 'breed'] = 'Redbone Hound Mix'
working_df.loc[working_df['breed'].str.contains('Dachshund Wirehair/'), 'breed'] = 'Dachshund Wirehair Mix'
working_df.loc[working_df['breed'].str.contains('Doberman Pinsch/'), 'breed'] = 'Doberman Pinsch Mix'
working_df.loc[working_df['breed'].str.contains('Flat Coat Retriever/'), 'breed'] = 'Flat Coat Retriever Mix'
working_df.loc[working_df['breed'].str.contains('Mastiff/'), 'breed'] = 'Mastiff Mix'
working_df.loc[working_df['breed'].str.contains('Shih Tzu/'), 'breed'] = 'Shih Tzu Mix'
working_df.loc[working_df['breed'].str.contains('Chow Chow/'), 'breed'] = 'Chow Chow Mix'
working_df.loc[working_df['breed'].str.contains('Carolina Dog/'), 'breed'] = 'Carolina Dog Mix'
working_df.loc[working_df['breed'].str.contains('Harrier/'), 'breed'] = 'Harrier Mix'
working_df.loc[working_df['breed'].str.contains('Toy Poodle/'), 'breed'] = 'Toy Poodle Mix'
working_df.loc[working_df['breed'].str.contains('Whippet/'), 'breed'] = 'Whippet Mix'
working_df.loc[working_df['breed'].str.contains('Chinese Sharpei/'), 'breed'] = 'Chinese Sharpei Mix'
working_df.loc[working_df['breed'].str.contains('Alaskan Husky/'), 'breed'] = 'Alaskan Husky Mix'
working_df.loc[working_df['breed'].str.contains('Pembroke Welsh Corgi/'), 'breed'] = 'Pembroke Welsh Corgi Mix'
working_df.loc[working_df['breed'].str.contains('Basenji/'), 'breed'] = 'Basenji Mix'
working_df.loc[working_df['breed'].str.contains('Rhod Ridgeback/'), 'breed'] = 'Rhod Ridgeback Mix'
working_df.loc[working_df['breed'].str.contains('Cocker Spaniel/'), 'breed'] = 'Cocker Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Great Dane/'), 'breed'] = 'Great Dane Mix'
working_df.loc[working_df['breed'].str.contains('Soft Coated Wheaten Terrier/'), 'breed'] = 'Soft Coated Wheaten Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Pomeranian/'), 'breed'] = 'Pomeranian Mix'
working_df.loc[working_df['breed'].str.contains('Bruss Griffon/'), 'breed'] = 'Bruss Griffon Mix'
working_df.loc[working_df['breed'].str.contains('Pug/'), 'breed'] = 'Pug Mix'
working_df.loc[working_df['breed'].str.contains('Norwich Terrier/'), 'breed'] = 'Norwich Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Belgian Malinois/'), 'breed'] = 'Belgian Malinois Mix'
working_df.loc[working_df['breed'].str.contains('Wire Hair Fox Terrier/'), 'breed'] = 'Wire Hair Fox Terrier Mix'
working_df.loc[working_df['breed'].str.contains('English Bulldog/'), 'breed'] = 'English Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Boston Terrier/'), 'breed'] = 'Boston Terrier Mix'
working_df.loc[working_df['breed'].str.contains('German Shorthair Pointer/'), 'breed'] = 'German Shorthair Pointer Mix'
working_df.loc[working_df['breed'].str.contains('Papillon/'), 'breed'] = 'Papillon Mix'
working_df.loc[working_df['breed'].str.contains('Tibetan Spaniel/'), 'breed'] = 'Tibetan Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Smooth Fox Terrier/'), 'breed'] = 'Smooth Fox Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Dalmatian/'), 'breed'] = 'Dalmatian Mix'
working_df.loc[working_df['breed'].str.contains('Pbgv/'), 'breed'] = 'Pbgv Mix'
working_df.loc[working_df['breed'].str.contains('Bull Terrier/'), 'breed'] = 'Bull Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Vizsla/'), 'breed'] = 'Vizsla Mix'
working_df.loc[working_df['breed'].str.contains('Standard Schnauzer/'), 'breed'] = 'Standard Schnauzer Mix'
working_df.loc[working_df['breed'].str.contains('Weimaraner/'), 'breed'] = 'Weimaraner Mix'
working_df.loc[working_df['breed'].str.contains('Scottish Terrier/'), 'breed'] = 'Scottish Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Dutch Shepherd/'), 'breed'] = 'Dutch Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('Pekingese/'), 'breed'] = 'Pekingese Mix'
working_df.loc[working_df['breed'].str.contains('Akita/'), 'breed'] = 'Akita Mix'
working_df.loc[working_df['breed'].str.contains('Pharaoh Hound/'), 'breed'] = 'Pharaoh Hound Mix'
working_df.loc[working_df['breed'].str.contains('Collie Rough/'), 'breed'] = 'Collie Rough Mix'
working_df.loc[working_df['breed'].str.contains('Bichon Frise/'), 'breed'] = 'Bichon Frise Mix'
working_df.loc[working_df['breed'].str.contains('English Coonhound/'), 'breed'] = 'English Coonhound Mix'
working_df.loc[working_df['breed'].str.contains('Italian Greyhound/'), 'breed'] = 'Italian Greyhound Mix'
working_df.loc[working_df['breed'].str.contains('Shiba Inu/'), 'breed'] = 'Shiba Inu Mix'
working_df.loc[working_df['breed'].str.contains('English Pointer/'), 'breed'] = 'English Pointer Mix'
working_df.loc[working_df['breed'].str.contains('Greyhound/'), 'breed'] = 'Greyhound Mix'
working_df.loc[working_df['breed'].str.contains('Lhasa Apso/'), 'breed'] = 'Lhasa Apso Mix'
working_df.loc[working_df['breed'].str.contains('Havanese/'), 'breed'] = 'Havanese Mix'
working_df.loc[working_df['breed'].str.contains('Shetland Sheepdog/'), 'breed'] = 'Shetland Sheepdog Mix'
working_df.loc[working_df['breed'].str.contains('West Highland/'), 'breed'] = 'West Highland Mix'
working_df.loc[working_df['breed'].str.contains('Bluetick Hound/'), 'breed'] = 'Bluetick Hound Mix'
working_df.loc[working_df['breed'].str.contains('Airedale Terrier/'), 'breed'] = 'Airedale Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Finnish Spitz/'), 'breed'] = 'Finnish Spitz Mix'
working_df.loc[working_df['breed'].str.contains('Chesa Bay Retr/'), 'breed'] = 'Chesa Bay Retr Mix'
working_df.loc[working_df['breed'].str.contains('Schnauzer Giant/'), 'breed'] = 'Schnauzer Giant Mix'
working_df.loc[working_df['breed'].str.contains('Bernese Mountain Dog/'), 'breed'] = 'Bernese Mountain Dog Mix'
working_df.loc[working_df['breed'].str.contains('Australian Terrier/'), 'breed'] = 'Australian Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Alaskan Malamute/'), 'breed'] = 'Alaskan Malamute Mix'
working_df.loc[working_df['breed'].str.contains('Parson Russell Terrier/'), 'breed'] = 'Parson Russell Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Toy Fox Terrier/'), 'breed'] = 'Toy Fox Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Irish Terrier/'), 'breed'] = 'Irish Terrier Mix'
working_df.loc[working_df['breed'].str.contains('American Eskimo/'), 'breed'] = 'American Eskimo Mix'
working_df.loc[working_df['breed'].str.contains('Bulldog/'), 'breed'] = 'Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Leonberger/'), 'breed'] = 'Leonberger Mix'
working_df.loc[working_df['breed'].str.contains('Swedish Vallhund/'), 'breed'] = 'Swedish Vallhund Mix'
working_df.loc[working_df['breed'].str.contains('Treeing Walker Coonhound/'), 'breed'] = 'Treeing Walker Coonhound Mix'
working_df.loc[working_df['breed'].str.contains('Glen Of Imaal/'), 'breed'] = 'Glen Of Imaal Mix'
working_df.loc[working_df['breed'].str.contains('Bloodhound/'), 'breed'] = 'Bloodhound Mix'
working_df.loc[working_df['breed'].str.contains('Bullmastiff/'), 'breed'] = 'Bullmastiff Mix'
working_df.loc[working_df['breed'].str.contains('Tibetan Terrier/'), 'breed'] = 'Tibetan Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Welsh Terrier/'), 'breed'] = 'Welsh Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Cavalier Span/'), 'breed'] = 'Cavalier Span Mix'
working_df.loc[working_df['breed'].str.contains('St. Bernard Smooth Coat/'), 'breed'] = 'St. Bernard Smooth Coat Mix'
working_df.loc[working_df['breed'].str.contains('Landseer/'), 'breed'] = 'Landseer Mix'
working_df.loc[working_df['breed'].str.contains('Dogo Argentino/'), 'breed'] = 'Dogo Argentino Mix'
working_df.loc[working_df['breed'].str.contains('Patterdale Terr/'), 'breed'] = 'Patterdale Terr Mix'
working_df.loc[working_df['breed'].str.contains('Field Spaniel/'), 'breed'] = 'Field Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Feist/'), 'breed'] = 'Feist Mix'
working_df.loc[working_df['breed'].str.contains('Beauceron/'), 'breed'] = 'Beauceron Mix'
working_df.loc[working_df['breed'].str.contains('Schipperke/'), 'breed'] = 'Schipperke Mix'
working_df.loc[working_df['breed'].str.contains('Dogue De Bordeaux/'), 'breed'] = 'Dogue De Bordeaux Mix'
working_df.loc[working_df['breed'].str.contains('Boykin Span/'), 'breed'] = 'Boykin Span Mix'
working_df.loc[working_df['breed'].str.contains('Standard Poodle/'), 'breed'] = 'Standard Poodle Mix'
working_df.loc[working_df['breed'].str.contains('Canaan Dog/'), 'breed'] = 'Canaan Dog Mix'
working_df.loc[working_df['breed'].str.contains('Silky Terrier/'), 'breed'] = 'Silky Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Presa Canario/'), 'breed'] = 'Presa Canario Mix'
working_df.loc[working_df['breed'].str.contains('Treeing Cur/'), 'breed'] = 'Treeing Cur Mix'
working_df.loc[working_df['breed'].str.contains('Podengo Pequeno/'), 'breed'] = 'Podengo Pequeno Mix'
working_df.loc[working_df['breed'].str.contains('German Wirehaired Pointer/'), 'breed'] = 'German Wirehaired Pointer Mix'
working_df.loc[working_df['breed'].str.contains('Newfoundland/'), 'breed'] = 'Newfoundland Mix'
working_df.loc[working_df['breed'].str.contains('St. Bernard Rough Coat/'), 'breed'] = 'St. Bernard Rough Coat Mix'
working_df.loc[working_df['breed'].str.contains('Old English Bulldog/'), 'breed'] = 'Old English Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Boerboel/'), 'breed'] = 'Boerboel Mix'
working_df.loc[working_df['breed'].str.contains('Irish Wolfhound/'), 'breed'] = 'Irish Wolfhound Mix'
working_df.loc[working_df['breed'].str.contains('Keeshond/'), 'breed'] = 'Keeshond Mix'
working_df.loc[working_df['breed'].str.contains('Kuvasz/'), 'breed'] = 'Kuvasz Mix'
working_df.loc[working_df['breed'].str.contains('Cane Corso/'), 'breed'] = 'Cane Corso Mix'
working_df.loc[working_df['breed'].str.contains('American Foxhound/'), 'breed'] = 'American Foxhound Mix'
working_df.loc[working_df['breed'].str.contains('English Shepherd/'), 'breed'] = 'English Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('English Foxhound/'), 'breed'] = 'English Foxhound Mix'
working_df.loc[working_df['breed'].str.contains('Chinese Crested/'), 'breed'] = 'Chinese Crested Mix'
working_df.loc[working_df['breed'].str.contains('Picardy Sheepdog/'), 'breed'] = 'Picardy Sheepdog Mix'
working_df.loc[working_df['breed'].str.contains('Skye Terrier/'), 'breed'] = 'Skye Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Akbash/'), 'breed'] = 'Akbash Mix'
working_df.loc[working_df['breed'].str.contains('Wirehaired Pointing Griffon/'), 'breed'] = 'Wirehaired Pointing Griffon Mix'
working_df.loc[working_df['breed'].str.contains('Affenpinscher/'), 'breed'] = 'Affenpinscher Mix'
working_df.loc[working_df['breed'].str.contains('Mexican Hairless/'), 'breed'] = 'Mexican Hairless Mix'
working_df.loc[working_df['breed'].str.contains('Bearded Collie/'), 'breed'] = 'Bearded Collie Mix'
working_df.loc[working_df['breed'].str.contains('Greater Swiss Mountain Dog/'), 'breed'] = 'Greater Swiss Mountain Dog Mix'
working_df.loc[working_df['breed'].str.contains('French Bulldog/'), 'breed'] = 'French Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Nova Scotia Duck Tolling Retriever/'), 'breed'] = 'Nova Scotia Duck Tolling Retriever Mix'
working_df.loc[working_df['breed'].str.contains('Neapolitan Mastiff/'), 'breed'] = 'Neapolitan Mastiff Mix'
working_df.loc[working_df['breed'].str.contains('German Pinscher/'), 'breed'] = 'German Pinscher Mix'
working_df.loc[working_df['breed'].str.contains('English Cocker Spaniel/'), 'breed'] = 'English Cocker Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Treeing Tennesse Brindle/'), 'breed'] = 'Treeing Tennesse Brindle Mix'
working_df.loc[working_df['breed'].str.contains('Ibizan Hound/'), 'breed'] = 'Ibizan Hound Mix'
working_df.loc[working_df['breed'].str.contains('English Setter/'), 'breed'] = 'English Setter Mix'
working_df.loc[working_df['breed'].str.contains('Bull Terrier Miniature/'), 'breed'] = 'Bull Terrier Miniature Mix'
working_df.loc[working_df['breed'].str.contains('Welsh Springer Spaniel/'), 'breed'] = 'Welsh Springer Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Coton De Tulear/'), 'breed'] = 'Coton De Tulear Mix'
working_df.loc[working_df['breed'].str.contains('Brittany/'), 'breed'] = 'Brittany Mix'
working_df.loc[working_df['breed'].str.contains('Dandie Dinmont/'), 'breed'] = 'Dandie Dinmont Mix'
working_df.loc[working_df['breed'].str.contains('Belgian Sheepdog/'), 'breed'] = 'Belgian Sheepdog Mix'
working_df.loc[working_df['breed'].str.contains('Port Water Dog/'), 'breed'] = 'Port Water Dog Mix'
working_df.loc[working_df['breed'].str.contains('Bedlington Terr/'), 'breed'] = 'Bedlington Terr Mix'
working_df.loc[working_df['breed'].str.contains('Gordon Setter/'), 'breed'] = 'Gordon Setter Mix'
working_df.loc[working_df['breed'].str.contains('Eng Toy Spaniel/'), 'breed'] = 'Eng Toy Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Dachshund Stan/'), 'breed'] = 'Dachshund Stan Mix'
working_df.loc[working_df['breed'].str.contains('English Springer Spaniel/'), 'breed'] = 'English Springer Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Clumber Spaniel/'), 'breed'] = 'Clumber Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Saluki/'), 'breed'] = 'Saluki Mix'
working_df.loc[working_df['breed'].str.contains('Grand Basset Griffon Vendeen/'), 'breed'] = 'Grand Basset Griffon Vendeen Mix'
working_df.loc[working_df['breed'].str.contains('Hovawart/'), 'breed'] = 'Hovawart Mix'
working_df.loc[working_df['breed'].str.contains('Samoyed/'), 'breed'] = 'Samoyed Mix'
working_df.loc[working_df['breed'].str.contains('Jindo/'), 'breed'] = 'Jindo Mix'
working_df.loc[working_df['breed'].str.contains('Lakeland Terrier/'), 'breed'] = 'Lakeland Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Lowchen/'), 'breed'] = 'Lowchen Mix'
working_df.loc[working_df['breed'].str.contains('Norwegian Elkhound/'), 'breed'] = 'Norwegian Elkhound Mix'
working_df.loc[working_df['breed'].str.contains('Otterhound/'), 'breed'] = 'Otterhound Mix'
working_df.loc[working_df['breed'].str.contains('Belgian Tervuren/'), 'breed'] = 'Belgian Tervuren Mix'
working_df.loc[working_df['breed'].str.contains('Sealyham Terr/'), 'breed'] = 'Sealyham Terr Mix'
working_df.loc[working_df['breed'].str.contains('Japanese Chin/'), 'breed'] = 'Japanese Chin Mix'
working_df.loc[working_df['breed'].str.contains('Afghan Hound/'), 'breed'] = 'Afghan Hound Mix'
working_df.loc[working_df['breed'].str.contains('Entlebucher/'), 'breed'] = 'Entlebucher Mix'

In [14]:
mix_df = working_df[working_df['breed'].str.contains('Mix')]\
        .sort_values(by=['animal_id'], ascending=False)
pure_df = working_df[~working_df['breed'].str.contains('Mix')]\
        .sort_values(by=['animal_id'], ascending=False)

In [48]:
def find_num_stays(df):
#Finds the number of stays for a particular animal id if and only if
#the stay was  followed by an adoption outcome.
    stay_list=[]
    first_row = df.iloc[0, :]
    if first_row["is_intake"]:
        x_len = len(df)
#The range has to be the length of the dataframe-1 because of the need to add 1 to the current row
#The for loop cannot use iterrows because the result has to refer to the current row and the next one.
        for i in range(0, x_len - 1):
#Sets variables equal to the first and second rows of the dataframe
            x1_row = df.iloc[i, :]
            x2_row = df.iloc[i + 1, :]
#First in pair has to be intake, the next has to be an outcome
            if (x1_row["is_intake"]==True) & (x2_row["is_intake"]==False) and (x1_row["animal_id"] == x2_row["animal_id"]):
#A pair of rows is valid if they are not both intakes or both outcomes.
                if x1_row["is_intake"] != x2_row["is_intake"]:
#Stay length is the number of days between intake and adoption outcome.
                    stay = (x2_row['datetime']-x1_row["datetime"]).days
                    stay_list.append(stay)
# Calculates and returns average
    num_stays=len(stay_list)
    return num_stays
find_num_stays(molly)

8

In [77]:
agg_stays_df = working_df[["animal_id", "is_intake", "datetime"]].groupby("animal_id").agg(find_num_stays)

In [95]:
agg_stays_df2 = agg_stays_df[agg_stays_df["is_intake"]!=0].reset_index()
working_df2 = working_df.loc[
    working_df["animal_id"]
    .isin(agg_stays_df2["animal_id"].values)
]
agg_stays_df2 = agg_stays_df2.set_index("animal_id", drop=True)

working_df2 = working_df2.set_index("animal_id", drop=True).assign(num_stays = agg_stays_df2["is_intake"]).reset_index()

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,is_intake,date_of_birth,outcome_subtype,color_primary,color_secondary,num_stays
37695,A754989,Molly,2017-07-27 14:54:00,5801 Signal Point Blvd in Austin (TX),Owner Surrender,Normal,Dog,Intact Female,1 month,Labrador Retriever Mix,True,N/A,N/A,Tan,None,8
37696,A754989,Molly,2017-07-29 18:39:00,N/A,Adoption,N/A,Dog,Spayed Female,2 months,Labrador Retriever Mix,False,05/27/2017,N/A,Tan,None,8
37697,A754989,Molly,2018-12-06 15:46:00,Manor (TX),Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,True,N/A,N/A,Tan,None,8
37698,A754989,Molly,2018-12-17 18:15:00,N/A,Adoption,N/A,Dog,Spayed Female,1 year,Labrador Retriever Mix,False,05/27/2017,N/A,Tan,None,8
37699,A754989,Molly,2019-01-06 15:52:00,Outside Jurisdiction,Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,True,N/A,N/A,Tan,None,8
37700,A754989,Molly,2019-01-18 19:17:00,N/A,Adoption,N/A,Dog,Spayed Female,1 year,Labrador Retriever Mix,False,05/27/2017,N/A,Tan,None,8
37701,A754989,Molly,2019-01-19 11:20:00,Outside Jurisdiction,Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,True,N/A,N/A,Tan,None,8
37702,A754989,Molly,2019-01-19 13:40:00,N/A,Adoption,N/A,Dog,Spayed Female,1 year,Labrador Retriever Mix,False,05/27/2017,N/A,Tan,None,8
37703,A754989,Molly,2019-02-18 11:00:00,Austin (TX),Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,True,N/A,N/A,Tan,None,8
37704,A754989,Molly,2019-02-19 19:04:00,N/A,Adoption,N/A,Dog,Spayed Female,1 year,Labrador Retriever Mix,False,05/27/2017,N/A,Tan,None,8


In [46]:
#Creates Molly dataframe for testing.
molly = working_df.set_index("animal_id").loc["A754989"].reset_index()
def find_staymean(df):
#Finds the mean length of stay for a particular animal id if and only if
#the stay has followed by an adoption outcome.
    stay_list=[]
    first_row = df.iloc[0, :]
    if first_row["is_intake"]:
        x_len = len(df)
#The range has to be the length of the dataframe-1 because of the need to add 1 to the current row
#The for loop cannot use iterrows because the result has to refer to the current row and the next one.
        for i in range(0, x_len - 1):
#Sets variables equal to the first and second rows of the dataframe
            x1_row = df.iloc[i, :]
            x2_row = df.iloc[i + 1, :]
#First in pair has to be intake, the next has to be an outcome
            if (x1_row["is_intake"]==True) & (x2_row["is_intake"]==False):
#A pair of rows is valid if they are not both intakes or both outcomes.
                if x1_row["is_intake"] != x2_row["is_intake"]:
#Stay length is the number of days between intake and adoption outcome.
                    stay = (x2_row['datetime']-x1_row["datetime"]).days
                    stay_list.append(stay)
# Calculates and returns average
    try:
        average_stay = sum(stay_list)/len(stay_list)
    except ZeroDivisionError:
        average_stay = 0
        
    return average_stay
find_staymean(molly)

7.5

In [96]:
stay_len_df = working_df2[["animal_id", "is_intake", "datetime"]].groupby("animal_id").agg(find_staymean)

,is_intake,datetime
animal_id,,
A200922,49.0,49.0
A210457,9.0,9.0
A226069,23.0,23.0
A249087,89.0,89.0
A274546,2.0,2.0
...,...,...
A811450,3.0,3.0
A811507,2.0,2.0
A811509,0.0,0.0


In [104]:
stay_len_df2 = stay_len_df[stay_len_df["is_intake"]!=0].reset_index()

working_df3 = working_df2.loc[
    working_df2["animal_id"]
    .isin(stay_len_df2["animal_id"].values)]

stay_len_df2 = stay_len_df2.set_index("animal_id", drop=True)

working_df3 = working_df3.set_index("animal_id", drop=True).assign(avg_stay_len = stay_len_df2["is_intake"]).reset_index()

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,is_intake,date_of_birth,outcome_subtype,color_primary,color_secondary,num_stays,avg_stay_len
0,A200922,Carlos,2013-10-03 15:47:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,16 years,Dachshund Mix,True,N/A,N/A,Black,Tan,1,49.0
1,A200922,Carlos,2013-11-22 09:44:00,N/A,Adoption,N/A,Dog,Neutered Male,16 years,Dachshund Mix,False,10/03/1997,Foster,Black,Tan,1,49.0
2,A210457,Caleb,2016-09-28 12:05:00,4424 S Mopac #412 in Austin (TX),Public Assist,Aged,Dog,Neutered Male,17 years,Chihuahua Shorthair,True,N/A,N/A,Tan,Black,1,9.0
3,A210457,Caleb,2016-10-07 12:34:00,N/A,Adoption,N/A,Dog,Neutered Male,17 years,Chihuahua Shorthair,False,06/01/1999,Foster,Tan,Black,1,9.0
4,A226069,Cedar,2015-10-06 12:29:00,Wheless Ln And Berkman Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,15 years,Labrador Retriever Mix,True,N/A,N/A,Tricolor,White,1,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60397,A811507,N/A,2020-01-06 17:37:00,N/A,Adoption,N/A,Dog,Neutered Male,2 months,Wire Hair Fox Terrier,False,10/20/2019,N/A,Black,White,1,2.0
60398,A811534,Lilu,2020-01-05 13:01:00,Austin (TX),Owner Surrender,Normal,Dog,Spayed Female,11 years,Dachshund,True,N/A,N/A,Black,Brown,1,2.0
60399,A811534,Lilu,2020-01-07 15:01:00,N/A,Adoption,N/A,Dog,Spayed Female,11 years,Dachshund,False,01/05/2009,N/A,Black,Brown,1,2.0
60400,A811598,Ginger,2020-01-06 12:09:00,Travis (TX),Owner Surrender,Normal,Dog,Spayed Female,8 years,Chihuahua Shorthair,True,N/A,N/A,White,Red,1,1.0


In [127]:
grouped_dogs_df = working_df3.groupby("animal_id").first()
describe_df2 = grouped_dogs_df.sort_values(by= "avg_stay_len")
describe_df2.head(10)

,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,is_intake,date_of_birth,outcome_subtype,color_primary,color_secondary,num_stays,avg_stay_len
animal_id,,,,,,,,,,,,,,,,
A803641,Pancho,2019-09-03 19:15:00,Austin (TX),Owner Surrender,Normal,Dog,Intact Male,2 months,Chihuahua Shorthair Mix,True,N/A,N/A,Brown,White,3,0.333333
A747956,Princess Ana,2017-04-27 12:39:00,Austin (TX),Owner Surrender,Normal,Dog,Intact Female,2 months,Chihuahua Shorthair Mix,True,N/A,N/A,Tan,None,2,0.500000
A747955,King Paco,2017-04-27 12:39:00,Austin (TX),Owner Surrender,Normal,Dog,Intact Male,2 months,Chihuahua Shorthair Mix,True,N/A,N/A,Black,Tan,2,0.500000
A563004,Coco,2014-01-04 12:41:00,1601 E Slaughter in Austin (TX),Owner Surrender,Normal,Dog,Spayed Female,5 years,Miniature Poodle,True,N/A,N/A,Black,Tan,2,0.500000
A803782,Kobe,2019-09-06 11:38:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,9 months,Cardigan Welsh Corgi Mix,True,N/A,N/A,Brown,White,2,0.500000
A695904,Indiana,2015-01-25 12:21:00,Del Valle (TX),Owner Surrender,Normal,Dog,Intact Male,1 month,Border Collie Mix,True,N/A,N/A,Tricolor,None,2,0.500000
A686364,Bubba,2014-08-20 18:01:00,Austin (TX),Owner Surrender,Normal,Dog,Intact Male,4 months,Pug Mix,True,N/A,N/A,Brown,Black,2,0.500000
A790716,*Caesar,2019-03-15 11:12:00,Manor (TX),Owner Surrender,Normal,Dog,Neutered Male,4 years,Alaskan Malamute,True,N/A,N/A,Tricolor,None,2,0.500000
A743015,Hershey,2017-02-02 09:06:00,Manor (TX),Owner Surrender,Normal,Dog,Spayed Female,1 year,Labrador Retriever Mix,True,N/A,N/A,Black,White,2,0.500000


In [141]:
dog_columns = list(grouped_dogs_df.columns)
for column in ["animal_type", "outcome_subtype", "is_intake", "date_of_birth"]:
    dog_columns.remove(column)
final_df = grouped_dogs_df[dog_columns].reset_index()


In [152]:
def round_off(avg_stay_len):
    if avg_stay_len <1:
        avg_stay_len = 1
    
    return avg_stay_len

final_df["avg_stay_len"] = final_df["avg_stay_len"].apply(round_off)

In [153]:
final_df.to_csv("Animal_Shelter_Final.csv", index=False)

In [41]:
# def construct_df_calc_stay_len(df):
#     stay_list=[]
#     working_df2=pd.DataFrame()
#     df["stay_len"]=np.nan
#     df["outcome_age"]=np.nan
#     df["outcome_dt"]=np.nan
#     df["num_stays"]=np.nan
# #Finds the number of stays for a particular animal id if and only if
# #the stay was  followed by an adoption outcome.
#     first_row = df.iloc[0, :]
#     if first_row["is_intake"]:
#         x_len = len(df)
# #The range has to be the length of the dataframe-1 because of the need to add 1 to the current row
# #The for loop cannot use iterrows because the result has to refer to the current row and the next one.
#         for i in range(0, x_len - 1):
# #Sets variables equal to the first and second rows of the dataframe
#             x1_row = df.iloc[i, :]
#             x2_row = df.iloc[i + 1, :]
#             x2_age = df.iat[i+1, 7]
#             x1_row["outcome_age"]=x2_age
#             x2_outcome_dt = df.iat[i+1, 1]
# #First in pair has to be intake, the next has to be an outcome
#             if ((x1_row["is_intake"]) & (x2_row["is_intake"]==False)) and (x1_row["animal_id"]==x2_row["animal_id"]):
# #A pair of rows is valid if they are not both intakes or both outcomes.
#                 if x1_row["is_intake"] != x2_row["is_intake"]:
# #Stay length is the number of days between intake and adoption outcome.
#                     x1_row["outcome_dt"]=x2_outcome_dt
#                     stay = (x2_row['datetime']-x1_row["datetime"]).days
#                     x1_row["stay_len"]=stay
#                     stay_list.append(stay)
#                     numstays=len(stay_list)
#                     x1_row["num_stays"]=numstays
#                     working_df2=working_df2.append(x1_row, ignore_index=False)
#     working_df2.rename(columns={'type':'intake_type'}, inplace=True)
#     return working_df2

In [31]:
# trying something a bit more hacky in order to determine length of stay
# import datetime

# diffs_df = working_df[["animal_id", "is_intake", "datetime"]].assign(date_diffs = working_df["datetime"].diff())

# diffs_df = diffs_df[
#     (diffs_df["date_diffs"]>=datetime.timedelta(days=0)) 
#     &
#     ([diffs_df["animal_id"].iloc[i] == diffs_df["animal_id"].iloc[i-1] for i in diffs_df.index])
# ]

# # return is a bit wonky -- need to validate the number of stays first before calculating the difference in stay lengths

In [15]:
# # make sure to install these packages before running:
# # pip install pandas
# # pip install sodapy

# import pandas as pd
# from sodapy import Socrata
# import requests

# app_token="3mubvsyw1rybaisquwqluss7zqf6c3nqxu5g4x56odov7ggexx"
# def get_soda_api_data(endpoint, app_token, count=1000, offset=0, return_df=True):
#     params = {'$$app_token': app_token, '$limit': count, '$offset': offset, }
    
#     results = []

#     while True:

#         try:
#             r = requests.get(endpoint, params=params)
#             rcontent = r.json()

#             if rcontent == []:
#                 break

#             results.append(rcontent)
#             offset += count 
#             params['$offset'] = offset

#         except HTTPError as err:

#             if err.response.status_code == '404':
#                 break
#             else:
#                 print(err.response.status_code)
    
#     if return_df:
#         results_df = pd.DataFrame()

#         for i in results:
#             results_df = results_df.append(pd.io.json.json_normalize(i))
        
#         return results_df
    
#     else:
#         return results
    
# endpoint="https://data.austintexas.gov/resource/wter-evkm.json"

# get_soda_api_data(endpoint, app_token)